In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

import pandas_profiling as pp

## Load Data

In [2]:
#dict for dtypes
dtypes = {'gender':'category', 'diabetes':'category','hypertension':'category','smoking history':'category',
          'stroke':'category' ,'heart disease':'int64'}

df_data = pd.read_csv('/data/MQIC_Patient_Data_100k_Sample.csv',
                      dtype=dtypes)

In [3]:
## Exploratory Data Analysis

In [4]:
# TODO: choose adequate plots for raw data to show 
pp.ProfileReport(df_data)

/opt/conda/lib/python3.7/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))
/opt/conda/lib/python3.7/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))
/opt/conda/lib/python3.7/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))


Number of variables,8
Number of observations,100000
Total Missing (%),7.7%
Total size in memory,2.8 MiB
Average record size in memory,29.0 B
Numeric,2
Categorical,5
Boolean,1
Date,0
Text (Unique),0
Rejected,0


## Data Cleaning

In [5]:
print('Data cleaning...')
entries_before = df_data.shape[0]
df_data.dropna(axis=0, how='any', inplace=True)
entries_after = df_data.shape[0]
dropped = entries_before - entries_after
print(f'{dropped} of {entries_before} lines dropped')

Data cleaning...
47825 of 100000 lines dropped


## Data Wrangling

In [6]:
#rearrange df
df_data = df_data[['gender','age','diabetes','hypertension','stroke',
                   'smoking history','BMI', 'heart disease']]
#input variables X
X = df_data[['gender','age','diabetes','hypertension','stroke',
                   'smoking history','BMI']]
#output variable y
y = df_data['heart disease']

## Data Preparation

In [7]:
categorial = df_data[['gender', 'diabetes', 'hypertension', 'stroke', 'smoking history']].values

#onehotencode categorical data, i.e. every col except age and BMI

# TODO: maybe drop one hot encoding. Is one hot encoding necessary?
#one hot encode  categorical cols (see categorial) --> prep for xgboost
encoded_X = None
for i in range(0, categorial.shape[1]):
    label_encoder = LabelEncoder()
    feature = label_encoder.fit_transform(categorial[:,i])
    feature = feature.reshape(categorial.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
    feature = onehot_encoder.fit_transform(feature)
    #add feature to encoded_X
    if encoded_X is None:
        encoded_X = feature
    else:
        encoded_X = np.concatenate((encoded_X, feature), axis=1)


#add 'age','BMI'
numeric_cols = ['age', 'BMI']
for col in numeric_cols:
    values = X[col].values
    values = values.reshape(values.shape[0],1)
    encoded_X = np.concatenate((encoded_X, values), axis=1)
    
#reshape output variables
reshaped_y = y.values
reshaped_y = reshaped_y.reshape(y.shape[0],) 
     
#split train and test data
X_train, X_test, y_train, y_test = train_test_split(encoded_X, reshaped_y, test_size = 0.2)
print(reshaped_y)

[1 0 1 ... 0 0 0]


## Model

In [8]:
#fit model
model = XGBClassifier()
model.fit(X_train, y_train)
print(model)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
Accuracy: 95.13%


## Feature Importance